In [2]:
import glob
import subprocess
control_bams=glob.glob("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL*")

In [4]:
read_counts = 
import os
from tqdm import tqdm


for b in tqdm(control_bams):
    bam=glob.glob(b+'/*/*.bam')

    if len(bam) ==1:      
        bam=bam[0]
        cmd = "samtools view -c %s"%bam
        count = subprocess.check_output(cmd,shell=True)
        read_counts[bam] = count

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [36:37<00:00, 48.82s/it]


In [17]:
top5_samples = sorted(read_counts.items(), key=lambda kv: int(kv[1].strip()),reverse=True)[:5]

In [20]:
sample_list = []
for sample, count in top5_samples:
    sample_list += [sample]

In [21]:
sample_list

['/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUFL733GX5/CTRL-NEUFL733GX5-7759-E/CTRL-NEUFL733GX5-7759-E_1.trim.nodup.no_chrM_MT.bam',
 '/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUAA485DZL/CTRL-NEUAA485DZL-7577-E/CTRL-NEUAA485DZL-7577-E_1.trim.nodup.no_chrM_MT.bam',
 '/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUPL878MTL/CTRL-NEUPL878MTL-6345-E/CTRL-NEUPL878MTL-6345-E_1.trim.nodup.no_chrM_MT.bam',
 '/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUDT762KUL/CTRL-NEUDT762KUL-6908-E/CTRL-NEUDT762KUL-6908-E_1.trim.nodup.no_chrM_MT.bam',
 '/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUMF089KLV/CTRL-NEUMF089KLV-6978-E/CTRL-NEUMF089KLV-6978-E_1.trim.nodup.no_chrM_MT.bam']

In [22]:
top5_samples

[('/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUFL733GX5/CTRL-NEUFL733GX5-7759-E/CTRL-NEUFL733GX5-7759-E_1.trim.nodup.no_chrM_MT.bam',
  b'101075350\n'),
 ('/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUAA485DZL/CTRL-NEUAA485DZL-7577-E/CTRL-NEUAA485DZL-7577-E_1.trim.nodup.no_chrM_MT.bam',
  b'93275846\n'),
 ('/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUPL878MTL/CTRL-NEUPL878MTL-6345-E/CTRL-NEUPL878MTL-6345-E_1.trim.nodup.no_chrM_MT.bam',
  b'85921434\n'),
 ('/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUDT762KUL/CTRL-NEUDT762KUL-6908-E/CTRL-NEUDT762KUL-6908-E_1.trim.nodup.no_chrM_MT.bam',
  b'79402490\n'),
 ('/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/2_bam/CTRL-NEUMF089KLV/CTRL-NEUMF089KLV-6978-E/CTRL-NEUMF089KLV-6978-E_1.trim.nodup.no_chrM_MT.bam',
  b'74345584\n')]

### merge peaks

In [4]:
top5_samples=["CTRL-NEUFL733GX5","CTRL-NEUAA485DZL","CTRL-NEUPL878MTL","CTRL-NEUDT762KUL","CTRL-NEUMF089KLV"]


In [23]:
import glob
from tqdm import tqdm
import pandas as pd
total_peaks=[]
for s in tqdm(top5_samples):
    peak = "/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/3_peaks/%s"%s
    peak=glob.glob(peak+"/*/*narrowPeak.gz")
    assert len(peak) == 1
    peak=peak[0]
    peak=pd.read_csv(peak,sep='\t',header=None)
    for i, row in peak.iterrows():
        chromo,start,end=row.values[:3]
        summit = row[9]
        summit = start + summit
        start = summit - 150
        end = summit + 150
        total_peaks += [[chromo,start,end]]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:32<00:00,  6.51s/it]


In [27]:
total_peaks=pd.DataFrame(total_peaks)

In [30]:
total_peaks.to_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/chrombpnet_input/all_peaks_pooled.bed",\
                  sep='\t',index=False,header=None)

In [36]:
merged_peaks=pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/chrombpnet_input/all_peaks_pooled.merged.bed",\
           sep='\t',header=None)

In [42]:
formatted_peaks=[]
for i, row in merged_peaks.iterrows():
    chromo,start,end = row.values
    new_row=[chromo,start,end,"-","-","-","-","-","-",(end-start)//2]
    formatted_peaks += [new_row]

In [44]:
formatted_peak=pd.DataFrame(formatted_peaks)

In [46]:
formatted_peak.to_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/answer_ALS/chrombpnet_input/merged_overlap_optimal_peaks.narrowPeak",\
                     index=False,
                     header=None,\
                     sep='\t')

